# Data Job Postings Analysis

In this case study, we will analyze a dataset of job postings for data science and analytics roles. The dataset contains various columns such as job title, company, location, salary, job description, and required skills. However, some columns may have mostly missing values, which we will need to handle during preprocessing. For example, most companies may prefer not to disclose salary information, resulting in a column with many null values.


In [33]:
import polars as pl

## Load Dataset


The dataset is uploaded to a GitHub repository for convenience. You can also access the original dataset on Hugging Face Datasets at [`lukebarousse/data_jobs`](https://huggingface.co/datasets/lukebarousse/data_jobs). The original CSV file is quite large (~230 MB) due to the large number of rows and text fields, but it has been converted to Parquet format for this case study, which significantly reduces the file size to ~30 MB while preserving all data.


In [34]:
df = pl.read_parquet(
    "https://github.com/bdi593/datasets/raw/refs/heads/main/data-jobs/data_jobs.parquet"
)
df

job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
str,str,str,str,str,bool,str,str,bool,bool,str,str,f64,f64,str,str,str
"""Senior Data Engineer""","""Senior Clinical Data Engineer …","""Watertown, CT""","""via Work Nearby""","""Full-time""",false,"""Texas, United States""","""2023-06-16 13:44:15""",false,false,"""United States""",null,null,null,"""Boehringer Ingelheim""",null,null
"""Data Analyst""","""Data Analyst""","""Guadalajara, Jalisco, Mexico""","""via BeBee México""","""Full-time""",false,"""Mexico""","""2023-01-14 13:18:07""",false,false,"""Mexico""",null,null,null,"""Hewlett Packard Enterprise""","""['r', 'python', 'sql', 'nosql'…","""{'analyst_tools': ['power bi',…"
"""Data Engineer""","""Data Engineer/Scientist/Analys…","""Berlin, Germany""","""via LinkedIn""","""Full-time""",false,"""Germany""","""2023-10-10 13:14:55""",false,false,"""Germany""",null,null,null,"""ALPHA Augmented Services""","""['python', 'sql', 'c#', 'azure…","""{'analyst_tools': ['dax'], 'cl…"
"""Data Engineer""","""LEAD ENGINEER - PRINCIPAL ANAL…","""San Antonio, TX""","""via Diversity.com""","""Full-time""",false,"""Texas, United States""","""2023-07-04 13:01:41""",true,false,"""United States""",null,null,null,"""Southwest Research Institute""","""['python', 'c++', 'java', 'mat…","""{'cloud': ['aws'], 'libraries'…"
"""Data Engineer""","""Data Engineer- Sr Jobs""","""Washington, DC""","""via Clearance Jobs""","""Full-time""",false,"""Sudan""","""2023-08-07 14:29:36""",false,false,"""Sudan""",null,null,null,"""Kristina Daniel""","""['bash', 'python', 'oracle', '…","""{'cloud': ['oracle', 'aws'], '…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Software Engineer""","""DevOps Engineer""","""Singapura""","""melalui Trabajo.org""","""Pekerjaan tetap""",false,"""Singapore""","""2023-03-13 06:16:16""",false,false,"""Singapore""",null,null,null,"""CAREERSTAR INTERNATIONAL PTE. …","""['bash', 'python', 'perl', 'li…","""{'os': ['linux', 'unix'], 'oth…"
"""Data Analyst""","""CRM Data Analyst""","""Bad Rodach, Jerman""","""melalui BeBee Deutschland""","""Pekerjaan tetap""",false,"""Germany""","""2023-03-12 06:18:18""",false,false,"""Germany""",null,null,null,"""HABA FAMILYGROUP""","""['sas', 'sas', 'sql', 'excel']""","""{'analyst_tools': ['sas', 'exc…"
"""Business Analyst""","""Commercial Analyst - Start Now""","""Malaysia""","""melalui Ricebowl""","""Pekerjaan tetap""",false,"""Malaysia""","""2023-03-12 06:32:36""",false,false,"""Malaysia""",null,null,null,"""Lendlease Corporation""","""['powerpoint', 'excel']""","""{'analyst_tools': ['powerpoint…"


### Check the Number of Rows

The number of rows and columns in the dataset can be checked using the `shape` attribute of the Polars DataFrame. The `shape` attribute returns a tuple containing the number of rows and columns in the DataFrame.


In [35]:
df.shape

(785741, 17)

### Check Schema


Check the schema to understand the data types of each column.


In [36]:
df.schema

Schema([('job_title_short', String),
        ('job_title', String),
        ('job_location', String),
        ('job_via', String),
        ('job_schedule_type', String),
        ('job_work_from_home', Boolean),
        ('search_location', String),
        ('job_posted_date', String),
        ('job_no_degree_mention', Boolean),
        ('job_health_insurance', Boolean),
        ('job_country', String),
        ('salary_rate', String),
        ('salary_year_avg', Float64),
        ('salary_hour_avg', Float64),
        ('company_name', String),
        ('job_skills', String),
        ('job_type_skills', String)])

## Preprocess Dataset


### Check Missing Values


Check the number of missing values.


In [37]:
df.null_count()

job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,1,1045,8,12667,0,0,0,0,0,49,752674,763738,775079,1,117037,117037


The output is in a "wide" format, which makes it difficult to compare the number of missing values across columns. We can transpose the output to make it easier to read and analyze.


In [38]:
df.null_count().transpose(include_header=True)

column,column_0
str,u32
"""job_title_short""",0
"""job_title""",1
"""job_location""",1045
"""job_via""",8
"""job_schedule_type""",12667
…,…
"""salary_year_avg""",763738
"""salary_hour_avg""",775079
"""company_name""",1


We can go one step further and only display columns with one or more missing values to focus our attention on the columns that require preprocessing.


In [39]:
(
    df.null_count()
    .transpose(include_header=True, header_name="column", column_names=["null_count"])
    .filter(pl.col("null_count") > 0)
)

column,null_count
str,u32
"""job_title""",1
"""job_location""",1045
"""job_via""",8
"""job_schedule_type""",12667
"""job_country""",49
…,…
"""salary_year_avg""",763738
"""salary_hour_avg""",775079
"""company_name""",1


While the output is helpful, the number of rows displayed in the output is limited to 10 rows by default by Polars. There are more columns with missing values that are not displayed in the output. To see all columns with missing values, we can temporarily set the maximum number of rows to display to a higher value, such as 50, to ensure that all columns with missing values are shown in the output.


In [40]:
# Display up to 50 rows of the null count table
with pl.Config(tbl_rows=50):
    display(
        (
            df.null_count()
            .transpose(
                include_header=True, header_name="column", column_names=["null_count"]
            )
            .filter(pl.col("null_count") > 0)
        )
    )

column,null_count
str,u32
"""job_title""",1
"""job_location""",1045
"""job_via""",8
"""job_schedule_type""",12667
"""job_country""",49
"""salary_rate""",752674
"""salary_year_avg""",763738
"""salary_hour_avg""",775079
"""company_name""",1


:::{tip} What does the `with` statement do in Python?

The `with` statement in Python is used to wrap the execution of a block of code with methods defined by a context manager. It is commonly used for resource management, such as opening files or managing database connections, ensuring that resources are properly released after their use, even if an error occurs.

In the code example above, the `with` statement is used to temporarily set the maximum number of rows displayed in the output to 50. This means that when the block of code inside the `with` statement is executed, 50 rows of the DataFrame will be shown in the output, regardless of how many rows are actually in the DataFrame. After the block of code is executed, the maximum number of rows displayed will return to its previous setting.

The default maximum number of rows displayed in Polars is 10, so using the `with` statement allows you to temporarily change this setting for a specific block of code without affecting the global configuration.

:::


### Drop Rows with Missing Job Title


There is one row with a null value in the `job_title` column. Filter the DataFrame to show only this row.


In [41]:
df.filter(df["job_title"].is_null())

job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
str,str,str,str,str,bool,str,str,bool,bool,str,str,f64,f64,str,str,str
"""Data Engineer""",null,null,"""via Jobs In France - Mustakbil…",null,false,"""Saint Lucia""","""2023-12-14 07:32:05""",false,false,null,null,null,null,null,null,null


The dataset has over 780,000 rows, but only one row has a null value in the `job_title` and the `company` column.


In [42]:
df.filter(pl.col("job_skills").is_not_null()).select(pl.col("job_skills")).row(0)

("['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']",)

### Parse `"job_skills"` Column


The `"job_skills"` column is stored as a string that looks like a list, but it is not actually a list data type.

We can check the data type of the `"job_skills"` column to confirm that it is currently stored as a string. The output should indicate that the data type of the `"job_skills"` column is `String`.


In [ ]:
df.filter(pl.col("job_skills").is_not_null()).select(pl.col("job_skills")).head(5)

job_skills
list[str]
"[""r"", ""python"", … ""tableau""]"
"[""python"", ""sql"", … ""jenkins""]"
"[""python"", ""c++"", … ""pytorch""]"
"[""bash"", ""python"", … ""git""]"
"[""python"", ""sql"", ""gcp""]"


In [43]:
df.schema["job_skills"]

String

To convert it to a list, we can use the `str.replace_all()` method to replace single quotes with double quotes, and then use the `str.json_decode()` method to parse the string as JSON. This will give us a proper list of skills for each job.

:::{tip} Why do we need to replace single quotes with double quotes before parsing the string as JSON?

The JSON format requires that string values be enclosed in double quotes. If the string uses single quotes, it will not be valid JSON and the `str.json_decode()` method will fail to parse it correctly. By replacing single quotes with double quotes, we ensure that the string conforms to the JSON format, allowing us to successfully decode it into a list data type.

:::


In [57]:
df = df.with_columns(
    pl.when(pl.col("job_skills").is_not_null())
    .then(
        pl.col("job_skills")
        .str.replace_all("'", '"')
        .str.json_decode(dtype=pl.List(pl.Utf8))
    )
    .otherwise(None)
    .alias("job_skills")
)

df.filter(pl.col("job_skills").is_not_null()).select(pl.col("job_skills")).head(5)

InvalidOperationError: expected String type, got: list[str]

Verify that the `"job_skills"` column has been successfully parsed as a list by checking the data type of the column. The output should indicate that the data type of the `"job_skills"` column is now a list.


In [45]:
df.schema["job_skills"]

List(String)

#### How many jobs require "Python" as a skill?

We can use the `list.contains()` method to filter the DataFrame for rows where the `"job_skills"` list contains the skill "Python".


In [ ]:
df.filter(pl.col("job_skills").list.contains("python")).select(
    pl.col("job_title"), pl.col("company_name"), pl.col("job_skills")
)

job_title,company_name,job_skills
str,str,list[str]
"""Data Analyst""","""Hewlett Packard Enterprise""","[""r"", ""python"", … ""tableau""]"
"""Data Engineer/Scientist/Analys…","""ALPHA Augmented Services""","[""python"", ""sql"", … ""jenkins""]"
"""LEAD ENGINEER - PRINCIPAL ANAL…","""Southwest Research Institute""","[""python"", ""c++"", … ""pytorch""]"
"""Data Engineer- Sr Jobs""","""Kristina Daniel""","[""bash"", ""python"", … ""git""]"
"""GCP Data Engineer""","""smart folks inc""","[""python"", ""sql"", ""gcp""]"
…,…,…
"""Data Engineer""","""Shamrock Trading Corporation""","[""nosql"", ""mongodb"", … ""git""]"
"""Data Engineer (f/m/d)""","""Heidelberg Materials""","[""python"", ""c#"", … ""terraform""]"
"""Senior Data Engineer""","""Pure App""","[""sql"", ""python"", … ""docker""]"


The `height` attribute can be used to count the number of rows that match this condition.

It is equivalent to using the `len()` function, or `.shape[0]` to get the number of rows in the filtered DataFrame.


In [50]:
df.filter(pl.col("job_skills").list.contains("python")).height

380909

### Parse `"job_type_skills"` column


Similar to the `"job_skills"` column, the `"job_type_skills"` column is also stored as a string that contains dictionaries (or Map data type in more broad terms).


In [58]:
df.select("job_type_skills").filter(pl.col("job_type_skills").is_not_null()).head(5)

job_type_skills
str
"""{'analyst_tools': ['power bi',…"
"""{'analyst_tools': ['dax'], 'cl…"
"""{'cloud': ['aws'], 'libraries'…"
"""{'cloud': ['oracle', 'aws'], '…"
"""{'cloud': ['gcp'], 'programmin…"


In [60]:
df.select("job_type_skills").filter(pl.col("job_type_skills").is_not_null()).row(0)[0]

"{'analyst_tools': ['power bi', 'tableau'], 'programming': ['r', 'python', 'sql', 'nosql']}"

We can check the data type of the `"job_type_skills"` column to confirm that it is currently stored as a string.


In [53]:
df.schema["job_type_skills"]

String